In [1]:
import onnx
import numpy as np
from qonnx.util.basic import qonnx_make_model
from finn.util.visualization import showInNetron,showSrc
import onnxruntime as rt
from qonnx.util.basic import qonnx_make_model
from onnx.helper import make_tensor_value_info, make_node, make_graph, make_model, make_tensor
from onnx import numpy_helper
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
import finn.core.onnx_exec as oxe
import torch
from brevitas.nn import QuantReLU, QuantIdentity
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline import RoundAndClipThresholds
from qonnx.core.datatype import DataType
from qonnx.transformation.qcdq_to_qonnx import QCDQToQuant
import finn.core.onnx_exec as oxe

In [2]:
from qonnx.transformation.base import Transformation
from qonnx.transformation.batchnorm_to_affine import BatchNormToAffine
from qonnx.transformation.general import (
    ConvertDivToMul,
    ConvertSubToAdd,
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
)
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.remove import RemoveIdentityOps

from finn.transformation.streamline.absorb import (
    Absorb1BitMulIntoConv,
    Absorb1BitMulIntoMatMul,
    AbsorbAddIntoMultiThreshold,
    AbsorbMulIntoMultiThreshold,
    AbsorbSignBiasIntoMultiThreshold,
    FactorOutMulSignMagnitude,
    AbsorbTransposeIntoMultiThreshold
)
from finn.transformation.streamline.collapse_repeated import (
    CollapseRepeatedAdd,
    CollapseRepeatedMul,
)
from finn.transformation.streamline.reorder import (
    MoveAddPastConv,
    MoveAddPastMul,
    MoveMulPastMaxPool,
    MoveScalarAddPastMatMul,
    MoveScalarLinearPastInvariants,
    MoveScalarMulPastConv,
    MoveScalarMulPastMatMul,
    MoveLinearPastEltwiseAdd,
    MoveLinearPastEltwiseMul
)
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.streamline.sign_to_thres import ConvertSignToThres

In [3]:
id_path = "./streamline_model_roundandclip.onnx"
model = ModelWrapper(id_path)
# showInNetron('./streamline_model_roundandclip.onnx')

In [4]:
# Block to extract all the parameters from the streamlined graph
streamlined_graph = onnx.load("./streamline_model_roundandclip.onnx")
# brevitas_lstm_export = onnx.load("./quant_lstm_quantization_qcdq.onnx")
parameters = streamlined_graph.graph.initializer #Extracting all the parameters from the loaded graph

# In this loop we will be printing all the parameters to correctly import the parameters values to the right variables
for i in range(len(parameters)):
    w = numpy_helper.to_array(parameters[i])
    print (streamlined_graph.graph.initializer[i].name)
    print(w.shape)
    print(i)
    # print(w,',',i)
    print("-------------------------")

Add_5_param0
(5, 1)
0
-------------------------
MatMul_3_param0
(20, 10)
1
-------------------------
MatMul_2_param0
(20, 10)
2
-------------------------
MatMul_0_param0
(20, 10)
3
-------------------------
MatMul_1_param0
(20, 10)
4
-------------------------
MatMul_7_param0
(20, 20)
5
-------------------------
MatMul_6_param0
(20, 20)
6
-------------------------
MatMul_4_param0
(20, 20)
7
-------------------------
MatMul_5_param0
(20, 20)
8
-------------------------
MatMul_8_param0
(64, 20)
9
-------------------------
MatMul_9_param0
(32, 64)
10
-------------------------
MatMul_10_param0
(5, 32)
11
-------------------------
MultiThreshold_0_param0
(1, 255)
12
-------------------------
MultiThreshold_1_param0
(1, 255)
13
-------------------------
MultiThreshold_2_param0
(1, 63)
14
-------------------------
Mul_2_param0
()
15
-------------------------
Mul_4_param0
()
16
-------------------------
MultiThreshold_7_param0
(1, 63)
17
-------------------------
MultiThreshold_9_param0
(1, 63)

In [6]:
#Loading weights of the parameter
mm_0_weights = numpy_helper.to_array(parameters[3])
mm_1_weights = numpy_helper.to_array(parameters[4])
mm_2_weights = numpy_helper.to_array(parameters[2])
mm_3_weights = numpy_helper.to_array(parameters[1])
mm_4_weights = numpy_helper.to_array(parameters[7])
mm_5_weights = numpy_helper.to_array(parameters[8])
mm_6_weights = numpy_helper.to_array(parameters[6])
mm_7_weights = numpy_helper.to_array(parameters[5])
mm_8_weights = numpy_helper.to_array(parameters[9])
mm_9_weights = numpy_helper.to_array(parameters[10])
mm_10_weights = numpy_helper.to_array(parameters[11])

mt_0_weights = numpy_helper.to_array(parameters[12])
mt_1_weights = numpy_helper.to_array(parameters[13])
mt_2_weights = numpy_helper.to_array(parameters[14])
mt_3_weights = numpy_helper.to_array(parameters[29])
mt_4_weights = numpy_helper.to_array(parameters[30])
mt_5_weights = numpy_helper.to_array(parameters[31])
mt_6_weights = numpy_helper.to_array(parameters[32])
mt_7_weights = numpy_helper.to_array(parameters[17])
mt_8_weights = numpy_helper.to_array(parameters[20])
mt_9_weights = numpy_helper.to_array(parameters[18])
mt_10_weights = numpy_helper.to_array(parameters[19])
mt_11_weights = numpy_helper.to_array(parameters[21])
mt_12_weights = numpy_helper.to_array(parameters[22])
mt_13_weights = numpy_helper.to_array(parameters[23])
mt_14_weights = numpy_helper.to_array(parameters[24])
mt_15_weights = numpy_helper.to_array(parameters[25])
mt_16_weights = numpy_helper.to_array(parameters[26])
mt_17_weights = numpy_helper.to_array(parameters[27])
mt_18_weights = numpy_helper.to_array(parameters[28])
mt_19_weights = numpy_helper.to_array(parameters[33])
mt_20_weights = numpy_helper.to_array(parameters[34])


#Weights export happens here.
#MatMul Weights
np.savetxt('./qlstm_streamlined_model_weights/mm_0_weights.txt', mm_0_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_1_weights.txt', mm_1_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_2_weights.txt', mm_2_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_3_weights.txt', mm_3_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_4_weights.txt', mm_4_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_5_weights.txt', mm_5_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_6_weights.txt', mm_6_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_7_weights.txt', mm_7_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_8_weights.txt', mm_8_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_9_weights.txt', mm_9_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mm_10_weights.txt', mm_10_weights, delimiter=',',fmt='%d')

#Multithreshold Weights
np.savetxt('./qlstm_streamlined_model_weights/mt_0_weights.txt', mt_0_weights, delimiter=',',fmt='%f')
np.savetxt('./qlstm_streamlined_model_weights/mt_1_weights.txt', mt_1_weights, delimiter=',',fmt='%f')
np.savetxt('./qlstm_streamlined_model_weights/mt_2_weights.txt', mt_2_weights, delimiter=',',fmt='%f')
np.savetxt('./qlstm_streamlined_model_weights/mt_3_weights.txt', mt_3_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_4_weights.txt', mt_4_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_5_weights.txt', mt_5_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_6_weights.txt', mt_6_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_7_weights.txt', mt_7_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_8_weights.txt', mt_8_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_9_weights.txt', mt_9_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_10_weights.txt', mt_10_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_11_weights.txt', mt_11_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_12_weights.txt', mt_12_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_13_weights.txt', mt_13_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_14_weights.txt', mt_14_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_15_weights.txt', mt_15_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_16_weights.txt', mt_16_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_17_weights.txt', mt_17_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_18_weights.txt', mt_18_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_19_weights.txt', mt_19_weights, delimiter=',',fmt='%d')
np.savetxt('./qlstm_streamlined_model_weights/mt_20_weights.txt', mt_20_weights, delimiter=',',fmt='%d')

#Scalar Mul Weights
# np.savetxt('./lstm_ids_streamlined_weights/mul_3_weights.txt', mul_3_weights, delimiter='',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/mul_4_weights.txt', mul_4_weights, delimiter=',',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/mul_5_weights.txt', mul_5_weights, delimiter='',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/mul_6_weights.txt', mul_6_weights, delimiter='',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/mul_7_weights.txt', mul_7_weights, delimiter=',',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/mul_8_weights.txt', mul_8_weights, delimiter='',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/mul_9_weights.txt', mul_9_weights, delimiter=',',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/mul_10_weights.txt', mul_10_weights, delimiter='',fmt='%f')

#Scalar Add Weights
# np.savetxt('./qlstm_streamlined_model_weights/add_5_weights.txt', add_5_weights, delimiter=',',fmt='%f')
# np.savetxt('./qlstm_streamlined_model_weights/add_6_weights.txt', add_6_weights, delimiter=',',fmt='%f')
# np.savetxt('./lstm_ids_streamlined_weights/add_7_weights.txt', add_7_weights, delimiter=',',fmt='%f')